# TWAS plot

In [ ]:
suppressMessages({
    library(dplyr)
    library(ggtext)
    library(ggplot2)
})

## Functions

In [ ]:
save_plots <- function(p, fn){
    for(ext in c('.png', '.pdf', '.svg')){
        ggsave(paste0(fn,ext), p, width=30, height=15, units="cm")
    }
}

## Prepare data

### Prepare GWAS

In [ ]:
pgc2_file = '/ceph/projects/v4_phase3_paper/inputs/sz_gwas/pgc2_clozuk/map_phase3/_m/libd_hg38_pgc2sz_snps.tsv'
gwas_data_load = data.table::fread(pgc2_file) %>% janitor::clean_names()

sig_data <- gwas_data_load %>% subset(p < 0.05)
notsig_data <- gwas_data_load %>% subset(p >= 0.05) %>% group_by(chr) %>% sample_frac(0.1)
gwas_data <- bind_rows(sig_data, notsig_data) %>% 
    select(chr, our_snp_id, rsid, snp, bp, or, se, p)
gwas_data %>% head(2)

### Preparing TWAS data

In [ ]:
twas_file = paste0('/ceph/projects/v4_phase3_paper/analysis/twas_ea/gene_weights', 
                  '/fusion_pgc2/summary_stats/_m/fusion_associations.txt')
twas_data_load <- data.table::fread(twas_file) %>% janitor::clean_names() %>%
    select(id, file, hsq, best_gwas_id, eqtl_id, model, twas_z, twas_p, fdr, bonferroni) %>%
    inner_join(gwas_data, by=c("best_gwas_id"="our_snp_id"))
twas_data_load %>% head(2)

In [ ]:
data_cum <- twas_data_load %>% group_by(chr) %>% summarise(max_bp = max(bp)) %>%
    mutate(bp_add = lag(cumsum(as.numeric(max_bp)), default=0)) %>% select(chr, bp_add)
twas_data <- twas_data_load %>% inner_join(data_cum, by="chr") %>% 
    mutate(bp_cum = bp + bp_add, group = ifelse(twas_z < 0, "Down", "Up"), 
           id=ifelse(id == "", as.character(file), as.character(id)))
twas_data %>% head(2)

### All TWAS, Caudate

### Plotting settings

In [ ]:
axis_set <- twas_data %>% group_by(chr) %>% summarize(center = mean(bp_cum))

In [ ]:
df = twas_data[twas_data$bonferroni < 0.05, ]
df = head(df[order(df$bonferroni), ], 10)
df

### Plot

In [ ]:
manhplot1 <- ggplot(twas_data, aes(x=bp_cum, y=twas_z)) + 
    geom_hline(yintercept = 0, color = "black", linetype = "solid") + 
    geom_point(alpha=0.75, size=1.25, aes(color=forcats::as_factor(chr))) +
    geom_point(data=twas_data[twas_data$fdr < 0.05, ], color="red", alpha=0.75) +
    scale_x_continuous(label = axis_set$chr, breaks = axis_set$center) + 
    scale_y_continuous(expand = c(0,0), limits = c(-15, 15)) + 
    scale_color_manual(values = rep(c("grey40","grey60"), unique(length(axis_set$chr)))) + 
    labs(x = NULL, y = "Z-score") +
    ggrepel::geom_text_repel(data=df, fontface = "bold", aes(x=bp_cum, y=twas_z, label=id)) +
    theme_minimal(base_size=20) + 
    theme(
        legend.position="none", panel.border=element_blank(), 
        panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
        axis.title.y=element_markdown(), axis.text.x=element_text(angle=60, size=8, vjust=0.5)
    )
manhplot1

In [ ]:
save_plots(manhplot1, 'caudateOnly_twas_manhattanplot')

### Overlapping tissue

In [ ]:
overlapping <- data.table::fread("../../_m/overlapping_tissue_twasList.txt") %>% janitor::clean_names() %>%
    select(id, file, chr_twas, our_snp_id, best_gwas_id, p, twas_z, twas_p, fdr) %>%
    arrange(fdr)
overlapping %>% head(10)

In [ ]:
dfx = overlapping %>% select(file) %>% inner_join(twas_data, by='file') %>%
    arrange(fdr)

manhplot2 <- ggplot(twas_data, aes(x=bp_cum, y=twas_z)) + 
    geom_hline(yintercept = 0, color = "black", linetype = "solid") + 
    geom_point(alpha=0.75, size=1.25, aes(color=forcats::as_factor(chr))) +
    geom_point(data=twas_data[twas_data$fdr < 0.05, ], color="red", alpha=0.75) +
    geom_point(data=dfx, color="blue", alpha=0.6) +
    scale_x_continuous(label = axis_set$chr, breaks = axis_set$center) + 
    scale_y_continuous(expand = c(0,0), limits = c(-15, 15)) + 
    scale_color_manual(values = rep(c("grey40","grey60"), unique(length(axis_set$chr)))) + 
    labs(x = NULL, y = "Z-score") +
    ggrepel::geom_text_repel(data=head(dfx, 10), fontface = "bold", aes(x=bp_cum, y=twas_z, label=id)) +
    theme_minimal(base_size=20) + 
    theme(
        legend.position="none", panel.border=element_blank(), 
        panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
        axis.title.y=element_markdown(), axis.text.x=element_text(angle=60, size=8, vjust=0.5)
    )
manhplot2

In [ ]:
save_plots(manhplot2, 'twas_manhattanplot')

In [ ]:
dfx = overlapping %>% select(file) %>% inner_join(twas_data, by='file') %>%
    arrange(fdr)

manhplot2 <- ggplot(twas_data, aes(x=bp_cum, y=twas_z)) + 
    geom_hline(yintercept = 0, color = "black", linetype = "solid") + 
    geom_point(alpha=0.75, size=1.25, aes(color=forcats::as_factor(chr))) +
    geom_point(data=twas_data[twas_data$fdr < 0.05, ], color="red", alpha=0.75) +
    geom_point(data=dfx, color="blue", alpha=0.6) +
    scale_x_continuous(label = axis_set$chr, breaks = axis_set$center) + 
    scale_y_continuous(expand = c(0,0), limits = c(-15, 15)) + 
    scale_color_manual(values = rep(c("grey40","grey60"), unique(length(axis_set$chr)))) + 
    labs(x = NULL, y = "Z-score") +
    #ggrepel::geom_text_repel(data=head(dfx, 10), fontface = "bold", aes(x=bp_cum, y=twas_z, label=id)) +
    theme_minimal(base_size=20) + 
    theme(
        legend.position="none", panel.border=element_blank(), 
        panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
        axis.title.y=element_markdown(), axis.text.x=element_text(angle=60, size=8, vjust=0.5)
    )
manhplot2
save_plots(manhplot2, 'twas_manhattanplot_noLabels')

### TWAS in GWAS loci, caudate

In [ ]:
df = twas_data %>% filter(bonferroni < 0.05, p > 5e-8) %>% 
    arrange(fdr) %>% head(10)
df

In [ ]:
manhplot1 <- ggplot(twas_data, aes(x=bp_cum, y=twas_z)) + 
    geom_hline(yintercept = 0, color = "black", linetype = "solid") + 
    geom_point(alpha=0.75, size=1.25, aes(color=forcats::as_factor(chr))) +
    geom_point(data=twas_data[twas_data$fdr < 0.05, ], color="red", alpha=0.75) +
    scale_x_continuous(label = axis_set$chr, breaks = axis_set$center) + 
    scale_y_continuous(expand = c(0,0), limits = c(-15, 15)) + 
    scale_color_manual(values = rep(c("grey40","grey60"), unique(length(axis_set$chr)))) + 
    labs(x = NULL, y = "Z-score") +
    ggrepel::geom_text_repel(data=df, fontface = "bold", aes(x=bp_cum, y=twas_z, label=id)) +
    theme_minimal(base_size=20) + 
    theme(
        legend.position="none", panel.border=element_blank(), 
        panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
        axis.title.y=element_markdown(), axis.text.x=element_text(angle=60, size=8, vjust=0.5)
    )
manhplot1

### Overlapping tissue in significant GWAS loci

In [ ]:
overlapping = overlapping %>% filter(p > 5e-8) %>% arrange(fdr)
overlapping %>% head(10)

In [ ]:
dfx = overlapping %>% select(file) %>% inner_join(twas_data, by='file') %>%
    arrange(fdr)

manhplot2 <- ggplot(twas_data, aes(x=bp_cum, y=twas_z)) + 
    geom_hline(yintercept = 0, color = "black", linetype = "solid") + 
    geom_point(alpha=0.75, size=1.25, aes(color=forcats::as_factor(chr))) +
    #geom_point(data=twas_data[twas_data$fdr < 0.05, ], color="red", alpha=0.75) +
    geom_point(data=dfx, color="blue", alpha=0.6) +
    scale_x_continuous(label = axis_set$chr, breaks = axis_set$center) + 
    scale_y_continuous(expand = c(0,0), limits = c(-15, 15)) + 
    scale_color_manual(values = rep(c("grey40","grey60"), unique(length(axis_set$chr)))) + 
    labs(x = NULL, y = "Z-score") +
    ggrepel::geom_text_repel(data=head(dfx, 10), fontface = "bold", aes(x=bp_cum, y=twas_z, label=id)) +
    theme_minimal(base_size=20) + 
    theme(
        legend.position="none", panel.border=element_blank(), 
        panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
        axis.title.y=element_markdown(), axis.text.x=element_text(angle=60, size=8, vjust=0.5)
    )
manhplot2

### Caudate only in significant GWAS loci

In [ ]:
caudate_only <- data.table::fread("../../_m/caudate_only_twasList_genes.txt") %>% janitor::clean_names() %>%
    select(id, file, chr_twas, our_snp_id, best_gwas_id, p, twas_z, twas_p, fdr) %>% arrange(fdr)
caudate_only %>% head(10)

In [ ]:
df2 = caudate_only %>% select(file) %>% inner_join(twas_data, by="file") %>% arrange(fdr)

manhplot3 <- ggplot(twas_data, aes(x=bp_cum, y=twas_z)) + 
    geom_hline(yintercept = 0, color = "black", linetype = "solid") + 
    geom_point(alpha=0.75, size=1.25, aes(color=forcats::as_factor(chr))) +
    #geom_point(data=twas_data[twas_data$fdr < 0.05, ], color="red", alpha=0.75) +
    #geom_point(data=dfx, color="blue", alpha=0.6) +
    geom_point(data=df2, color="green", alpha=0.75) +
    scale_x_continuous(label = axis_set$chr, breaks = axis_set$center) + 
    scale_y_continuous(expand = c(0,0), limits = c(-15, 15)) + 
    scale_color_manual(values = rep(c("grey40","grey60"), unique(length(axis_set$chr)))) + 
    labs(x = NULL, y = "Z-score") +
    ggrepel::geom_text_repel(data=head(df2, 10), fontface = "bold", aes(x=bp_cum, y=twas_z, label=id)) +
    theme_minimal(base_size=20) + 
    theme(
        legend.position="none", panel.border=element_blank(), 
        panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
        axis.title.y=element_markdown(), axis.text.x=element_text(angle=60, size=8, vjust=0.5)
    )
manhplot3

### No labels

In [ ]:
manhplot0 <- ggplot(twas_data, aes(x=bp_cum, y=twas_z)) + 
    geom_hline(yintercept = 0, color = "black", linetype = "solid") + 
    geom_point(alpha=0.75, size=1.25, aes(color=forcats::as_factor(chr))) +
    geom_point(data=twas_data[twas_data$fdr < 0.05, ], color="red", alpha=0.75) +
    geom_point(data=dfx, color="blue", alpha=0.6) +
    geom_point(data=df2, color="green", alpha=0.75) +
    scale_x_continuous(label = axis_set$chr, breaks = axis_set$center) + 
    scale_y_continuous(expand = c(0,0), limits = c(-15, 15)) + 
    scale_color_manual(values = rep(c("grey40","grey60"), unique(length(axis_set$chr)))) + 
    labs(x = NULL, y = "Z-score") +
    #ggrepel::geom_text_repel(data=head(df2, 10), fontface = "bold", aes(x=bp_cum, y=twas_z, label=id)) +
    theme_minimal(base_size=20) + 
    theme(
        legend.position="none", panel.border=element_blank(), 
        panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
        axis.title.y=element_markdown(), axis.text.x=element_text(angle=60, size=8, vjust=0.5)
    )
manhplot0

## Save plots

In [ ]:
save_plots(manhplot0, 'noLabel_twas_manhattanplot')
save_plots(manhplot1, 'topSig_twas_manhattanplot')
save_plots(manhplot2, 'sharedTop_twas_manhattanplot')
save_plots(manhplot3, 'caudateTop_twas_manhattanplot')